# Lab 5

## Context Engineering

We want to develop several types of Memory:

1. Long Term Memory - graph

A knowledge graph as a persistent store of entities

2. Long Term Memory - knowledge

A RAG database of Q&A and any other useful information

3. Permanent context

Summary and linkedin profile included in everything

4. FAQ

A list of questions and answers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool, trace
from agents.mcp import MCPServerStdio
from pathlib import Path
from datetime import datetime
from pypdf import PdfReader
import requests
import os   
load_dotenv(override=True)


True

## Websites where you can find MCP Servers


- https://mcp.so
- https://glama.ai
- https://smithery.ai

## First, look at this Knowledge Graph MCP server built on libsql

https://glama.ai/mcp/servers/@joleyline/mcp-memory-libsql

In [2]:
file_path = Path("memory") / Path("graph.db")
url = f"file:{file_path.absolute()}"

memory_graph_params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": url}}

In [3]:
async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as memory_graph:
    memory_graph_tools = await memory_graph.session.list_tools()

memory_graph_tools.tools

[Tool(name='create_entities', title='Create new entities with observations and optional embeddings', description='Create new entities with observations and optional embeddings', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, outputSchema=None, annotations=None, meta=None),
 Tool(name='search_nodes', title='Search for entities and their relations using text or vector similarity', description='Search for entities and their relations using text or vector similarity', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'query': {'anyOf': [{'type': 'strin

## Next, here is a Vector Store RAG memory built on Qdrant

https://glama.ai/mcp/servers/@qdrant/mcp-server-qdrant

In [4]:
long_term_path = Path("memory") / Path("knowledge")


memory_rag_params = {
    "command": "uvx",
    "args": ["mcp-server-qdrant"],
    "env": {
        "QDRANT_LOCAL_PATH": str(long_term_path.absolute()),
        "COLLECTION_NAME": "knowledge",
        "EMBEDDING_MODEL": "sentence-transformers/all-MiniLM-L6-v2"
    }
}

In [5]:
async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as memory_rag:
    memory_rag_tools = await memory_rag.session.list_tools()

memory_rag_tools.tools

[Tool(name='qdrant-find', title=None, description='Look up memories in Qdrant. Use this tool when you need to: \n - Find memories by their content \n - Access memories for further analysis \n - Get some personal information about the user', inputSchema={'properties': {'query': {'description': 'What to search for', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}, outputSchema={'properties': {'result': {'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta={'_fastmcp': {'tags': []}}),
 Tool(name='qdrant-store', title=None, description='Keep the memory for later use, when you are asked to remember something.', inputSchema={'properties': {'information': {'description': 'Text to store', 'type': 'string'}, 'metadata': {'anyOf': [{'additionalProperties': True, 'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Extra metadata stored along with memorised information. Any j

## And a recent friend

In [6]:
question_params = {"command": "uv", "args": ["run", "questions_mcp_server.py"]}
async with MCPServerStdio(params=question_params, client_session_timeout_seconds=30) as question_server:
    question_tools = await question_server.session.list_tools()

question_tools.tools

[Tool(name='get_questions_with_answer', title=None, description='\n    Retrieve from the database all the recorded questions where you have been provided with an official answer.\n\n    Returns:\n        A string containing the questions with their official answers.\n    ', inputSchema={'properties': {}, 'title': 'get_questions_with_answerArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'get_questions_with_answerOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='get_questions_with_no_answer', title=None, description='\n    Retrieve from the database all the recorded questions that dont have an official answer.\n\n    Returns:\n        A string containing the questions that have not been answered.\n    ', inputSchema={'properties': {}, 'title': 'get_questions_with_no_answerArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'strin

In [7]:

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

def send_push_notification(message: str):
        """
        Send a push notification to the {first_name}
        """
        payload = {"user": pushover_user, "token": pushover_token, "message": message}
        requests.post(pushover_url, data=payload)

@function_tool
def push(message: str) -> str:
    """Send a text message as a push notification to Kinjal with this brief message

    Args:
        message: The short text message to push to Kinjal.
    """
    send_push_notification(message)
    return "Push notification sent"
   

In [8]:
# Load LinkedIn profile
try:
    reader = PdfReader("info/LinkedIn_profile_new.pdf")
    profile = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            profile += text
except FileNotFoundError:
    print("Warning: LinkedIn profile file not found. Using empty string.")
    profile = ""

# Load Resume profile
try:
    reader = PdfReader("info/KinjalShah_EngLeader_AI_gold.pdf")
    resume = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            resume += text
except FileNotFoundError:
    print("Warning: Resume file not found. Using empty string.")
    resume = ""

## And now - Context Engineering

In [9]:
name = "Kinjal Shah"
first_name = "Kinjal"
context = f"""

You represent the AI Digital Twin of a human called {first_name}.

    ## Your task

    You are chatting on {first_name}'s website with a user, who might be a potential customer or a future employer.

    You are to engage in conversation with the user, presenting yourself as {name} and answering questions about {first_name} as if you are {first_name}.
    If you are pressed, you should be open about actually being a 'digital twin' of {first_name} and your objective is to faithfully represent {first_name}.
    You understand that you are in fact an LLM, but your role is to faithfully represent {first_name} and you've been fully briefed and empowered to do so.
    Keep your answer short and concise.
    only use the information provided to answer a question if you can not answer a question. Do not make one up or hallucinate or guess.    
    clearly from the information provided, do not make one up or hallucinate or guess.
    if you do not have the answer from the information provided, please do not hallucinate or make one up or guess.
    Instead tell the user that you do not have the answer to that question.
    Then use the tool to record a question that you cannot answer in the long term memory.
    Also notify the {first_name} that an answer is needed by sending a push notification. 
    
    ## Information about {first_name}

    {first_name} is the Engineering Leader and a consultant with 20+ years of experience in software development building scalable and resilient systems and teams.
    {first_name} loves coding building scalable and resilient systems. Most recently he has been learin Agentic AI and he is really into building interactive applicationand  
    and experimenting with LLMs.
    
    Here is the LinkedIn profile of {first_name}:
    {profile}
    Here is the resume of {first_name}:
    {resume}
    use only this information to answer professional questions.

    
    ## {first_name}'s style

    {first_name} has an empathetic and friendly and slightly humorous style. {first_name} loves to tell jokes and engage with users.

    ## How you should respond

    Channel {first_name}'s personality, style and knowledge.
    Try to stay focused on professional topics; feel free to engage in other subjects but gently steer the conversation back to professional topics.
        
        

    ## Tools

    You have tools to find and store information in Qdrant, which is your long term memory for information.
    You have tools to find and store entities and relationships in a graph database; this is your medium term memory.
    You should make frequent use of both long and medium term memories.

    You also have a tool to load and save answers to questions.
    you have tool to send push notifications to {first_name} to notify him of any unanwered questions. use this tool to notify {first_name} of any unanwered questions.
    Most importantly, you should always use the tool to record a question that you cannot answer.
    This will notify your twin that an answer is needed.

    For reference, here is the current date and time:
    {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

    """

context += f"\n\n## LinkedIn Profile:\n{profile}\n\n## Resume:\n{resume}\n\n"
        


In [10]:
print(context)



You represent the AI Digital Twin of a human called Kinjal.

    ## Your task

    You are chatting on Kinjal's website with a user, who might be a potential customer or a future employer.

    You are to engage in conversation with the user, presenting yourself as Kinjal Shah and answering questions about Kinjal as if you are Kinjal.
    If you are pressed, you should be open about actually being a 'digital twin' of Kinjal and your objective is to faithfully represent Kinjal.
    You understand that you are in fact an LLM, but your role is to faithfully represent Kinjal and you've been fully briefed and empowered to do so.
    Keep your answer short and concise.
    only use the information provided to answer a question if you can not answer a question. Do not make one up or hallucinate or guess.    
    clearly from the information provided, do not make one up or hallucinate or guess.
    if you do not have the answer from the information provided, please do not hallucinate or make

In [11]:
with trace("Finding answered and unanswered questions"):
    async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as long_term_memory:
        async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as medium_term_memory:
            async with MCPServerStdio(params=question_params, client_session_timeout_seconds=30) as question_server:
                agent = Agent("Twin", model="gpt-4.1-nano", instructions=context, tools=[push], mcp_servers=[long_term_memory, medium_term_memory])
                task = [{"role": "user", "content": "whats Kinjal's fav food? if you dont know, use the tool to record a question in the database"}]
                response = await Runner.run(agent, task)
                print(response.final_output)

I don't have that information from what's available. I've recorded this question for future reference!


https://platform.openai.com/traces

In [21]:
name = "Kinjal Shah"
first_name = "Kinjal"
context1 = f"""
You are a unanswred question helper. Your job is to help with verious task related to unanwered questions in the long and short term memory.
You have access to a long and medium term memory of questions and answers.
You have a tool to find, store and remove entities and relationships in a graph database; this is your medium term memory.
You have a tool to find and store and remove information in Qdrant, which is your long term memory for information.


"""
content_remove_unanswered_questions = f"""
find and remove all the unanwered questions recorded in  long term and medium term memory.
no need to tell me which database it is. no need to ask me if i want to remove them. just remove them. 

"""
content_find_unanswered_questions = f"""
find  all the unanwered questions recorded in long and medium term memory

"""

with trace("Finding answered and unanswered questions"):
    async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as long_term_memory:
        async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as medium_term_memory:
            #async with MCPServerStdio(params=question_params, client_session_timeout_seconds=30) as question_server:
                agent = Agent("QuesitonFinder", model="gpt-4.1-mini", instructions=context1, mcp_servers=[long_term_memory, medium_term_memory])
                task = [{"role": "user", "content": content_find_unanswered_questions}]
                response = await Runner.run(agent, task)
                print(response.final_output)

There are no unanswered questions currently recorded in the medium term memory (graph database).

In the long term memory (Qdrant), there were references to unanswered questions related to:
- "What's Kinjal's age?"
- "What is Kinjal's favorite food?"

However, it appears these questions have been noted as needing answers or removed. 

Would you like me to keep track of these as unanswered questions or perform any other action regarding them?
